In [135]:
# Data file Electricity_20-09-2024.csv contains information about hourly electricity consumption (column Energy (kWh)) and Temperature.
# Another file sahkon-hinta-010121-240924.csv contains information about hourly electricity prices.

# First complete the following tasks

# - Change time format of both files to Pandas datetime

import pandas as pd


# Load the CSV file again with the correct delimiter
df_electricity = pd.read_csv("./Electricity_20-09-2024.csv", delimiter=';')
df_price = pd.read_csv("./sahkon-hinta-010121-240924.csv")

# Rename the columns for easier access and clean-up
df_electricity.columns = [
    'Time', 'Energy (kWh)', 'Energy night (kWh)', 'Energy day (kWh)', 'Temperature']

df_electricity['Energy (kWh)'] = df_electricity['Energy (kWh)'].str.replace(
    ',', '.').astype(float)
df_electricity['Energy night (kWh)'] = df_electricity['Energy night (kWh)'].str.replace(
    ',', '.').astype(float)
df_electricity['Energy day (kWh)'] = df_electricity['Energy day (kWh)'].str.replace(
    ',', '.').astype(float)
df_electricity['Temperature'] = df_electricity['Temperature'].str.replace(
    ',', '.').astype(float)

df_price.columns = ['Time', 'Price(cent/kWh)']

# Convert 'Time' column to datetime
df_electricity['Time'] = pd.to_datetime(pd.to_datetime(
    df_electricity['Time'].str.strip(), format='%d.%m.%Y %H:%M', errors='coerce'))

df_price['Time'] = pd.to_datetime(df_price['Time'], format='%d-%m-%Y %H:%M:%S')


# Merge the data frames on the 'Time' column
df_merged = pd.merge(df_electricity, df_price, on='Time', how='inner')

# # Calculate the hourly bill
df_merged['Hourly Bill (€)'] = (df_merged['Energy (kWh)'] *
                                (df_merged['Price(cent/kWh)'] / 100)).round(2)


# Convert 'Time' column to datetime for proper grouping
df_merged['Time'] = pd.to_datetime(df_merged['Time'])


# # Add columns for grouping (day, week, month)
# df_merged['Day'] = df_merged['Time'].dt.day
# # df_merged['Day'] = df_merged['Time'].dt.strftime('%Y-%m-%d')
# df_merged['Week'] = df_merged['Time'].dt.isocalendar().week
# df_merged['Year'] = df_merged['Time'].dt.isocalendar().year
# df_merged['Month'] = df_merged['Time'].dt.month

display(df_merged.head(10))

,Time,Energy (kWh),Energy night (kWh),Energy day (kWh),Temperature,Price(cent/kWh),Hourly Bill (€)
0,2021-01-01 00:00:00,1.988,1.988,NaN,-0.9,3.094,0.06
1,2021-01-01 01:00:00,1.462,1.462,NaN,-1.3,3.019,0.04
2,2021-01-01 02:00:00,1.101,1.101,NaN,-1.4,2.974,0.03
3,2021-01-01 03:00:00,1.031,1.031,NaN,-1.9,2.941,0.03
4,2021-01-01 04:00:00,1.044,1.044,NaN,-1.9,2.943,0.03
5,2021-01-01 05:00:00,1.284,1.284,NaN,-2.1,2.983,0.04
6,2021-01-01 06:00:00,1.985,1.985,NaN,-2.3,3.031,0.06
7,2021-01-01 07:00:00,1.587,NaN,1.587,-2.1,3.091,0.05
8,2021-01-01 08:00:00,1.199,NaN,1.199,-2.2,3.088,0.04
9,2021-01-01 09:00:00,0.678,NaN,0.678,-2.6,3.109,0.02


In [125]:
# Group by daily, weekly, and monthly using resample
daily_hourly_bill = df_merged.groupby(pd.Grouper(key='Time', freq='d')).agg({
    'Energy (kWh)': 'sum',
    'Hourly Bill (€)': 'sum',
    'Price(cent/kWh)': 'mean',
    'Temperature': 'mean'
}).reset_index()

# Display the results grouped by Year-Month-Week
print("Daily Consumption and Hourly Bill in Euro of each day:")

print(daily_hourly_bill)

Daily Consumption and Hourly Bill in Euro of each day:
           Time  Energy (kWh)  Hourly Bill (€)  Price(cent/kWh)  Temperature
0    2021-01-01        36.701             1.19         3.255167    -3.154167
1    2021-01-02        43.606             1.78         4.068750    -5.029167
2    2021-01-03        59.927             1.96         3.202458    -9.512500
3    2021-01-04        77.953             4.17         5.376833   -17.733333
4    2021-01-05        74.834             4.29         5.863458   -15.966667
...         ...           ...              ...              ...          ...
1356 2024-09-18        22.980             1.44         6.594417    12.437500
1357 2024-09-19        25.537             2.09         7.115625    14.195833
1358 2024-09-20        25.451             1.60         9.101792    13.033333
1359 2024-09-21        23.151             1.10         2.683917     9.029167
1360 2024-09-22         2.126             0.20        12.233750     4.445833

[1361 rows x 5 colum

In [123]:
weekly_hourly_bill = df_merged.groupby(pd.Grouper(key='Time', freq='W')).agg({
    'Energy (kWh)': 'sum',
    'Hourly Bill (€)': 'sum',
    'Price(cent/kWh)': 'mean',
    'Temperature': 'mean'
}).reset_index()

# Display the results grouped by Year-Month-Week
print("Weekly Consumption and Hourly Bill in Euro for every week:")

print(weekly_hourly_bill)

Weekly Consumption and Hourly Bill in Euro for every week:
          Time  Energy (kWh)  Hourly Bill (€)  Price(cent/kWh)  Temperature
0   2021-01-03       140.234             4.93         3.508792    -5.898611
1   2021-01-10       493.781            31.54         6.615589   -12.223810
2   2021-01-17       541.564            39.11         7.240702   -14.236310
3   2021-01-24       483.176            27.68         5.856375    -7.154167
4   2021-01-31       480.021            32.51         6.912179    -6.983333
..         ...           ...              ...              ...          ...
190 2024-08-25       183.244             1.44         0.754625    15.472024
191 2024-09-01       114.578             0.56         0.826893    15.085119
192 2024-09-08       176.551             6.55         5.606542    15.952381
193 2024-09-15       141.662             9.32         9.133482    14.388690
194 2024-09-22       144.462            13.29        10.767696    10.519048

[195 rows x 5 columns]


In [124]:
monthly_hourly_bill = df_merged.groupby(pd.Grouper(key='Time', freq='ME')).agg({
    'Energy (kWh)': 'sum',
    'Hourly Bill (€)': 'sum',
    'Price(cent/kWh)': 'mean',
    'Temperature': 'mean'
}).reset_index()

# Display the results grouped by Year-Month-Week
print("Monthly Consumption and Hourly Bill in Euro of each Month:")

print(monthly_hourly_bill)

Monthly Consumption and Hourly Bill in Euro of each Month:
         Time  Energy (kWh)  Hourly Bill (€)  Price(cent/kWh)  Temperature
0  2021-01-31      2138.776           135.77         6.351622    -9.738038
1  2021-02-28      1999.589           143.11         7.084027   -12.777381
2  2021-03-31      1554.240            74.60         4.755227    -3.680619
3  2021-04-30      1060.946            45.81         4.558654     2.035833
4  2021-05-31       960.209            52.29         5.696895     7.734812
5  2021-06-30       487.973            32.43         6.963349    16.182361
6  2021-07-31       662.879            58.21         9.765914    18.566935
7  2021-08-31       686.614            54.49         8.456485    13.500672
8  2021-09-30      1020.821           106.75        11.069868     7.390694
9  2021-10-31      1249.997            94.58         8.038060     4.971486
10 2021-11-30      1660.833           197.41        10.651190    -2.793750
11 2021-12-31      1664.150           418

In [136]:
start_time = '2022-01-01'
end_time = '2024-06-01'

# Filter the data frame based on the selected time interval
df_filtered = df_merged[(df_merged['Time'] >= start_time)
                        & (df_merged['Time'] <= end_time)]

display(df_filtered.head(10))

,Time,Energy (kWh),Energy night (kWh),Energy day (kWh),Temperature,Price(cent/kWh),Hourly Bill (€)
8762,2022-01-01 00:00:00,4.683,4.683,NaN,-8.8,5.778,0.27
8763,2022-01-01 01:00:00,3.618,3.618,NaN,-9.7,5.125,0.19
8764,2022-01-01 02:00:00,1.258,1.258,NaN,-10.0,5.230,0.07
8765,2022-01-01 03:00:00,1.128,1.128,NaN,-10.0,5.502,0.06
8766,2022-01-01 04:00:00,2.014,2.014,NaN,-10.0,4.671,0.09
8767,2022-01-01 05:00:00,1.917,1.917,NaN,-11.8,4.923,0.09
8768,2022-01-01 06:00:00,1.033,1.033,NaN,-12.2,5.033,0.05
8769,2022-01-01 07:00:00,1.475,NaN,1.475,-12.2,5.364,0.08
8770,2022-01-01 08:00:00,1.432,NaN,1.432,-12.4,6.158,0.09
8771,2022-01-01 09:00:00,1.516,NaN,1.516,-12.3,8.120,0.12


In [142]:
# total Consumption over selected period


print('Total Consumption over selected period:')

print(monthly_consumption)

Total Consumption over selected period:
         Time  Energy (kWh)
0  2022-01-31  2.365686e+07
1  2022-02-28  2.136749e+07
2  2022-03-31  2.362506e+07
3  2022-04-30  2.289373e+07
4  2022-05-31  2.365686e+07
5  2022-06-30  2.289373e+07
6  2022-07-31  2.365686e+07
7  2022-08-31  2.365686e+07
8  2022-09-30  2.289373e+07
9  2022-10-31  2.375225e+07
10 2022-11-30  2.289373e+07
11 2022-12-31  2.365686e+07
12 2023-01-31  2.365686e+07
13 2023-02-28  2.136749e+07
14 2023-03-31  2.362506e+07
15 2023-04-30  2.289373e+07
16 2023-05-31  2.365686e+07
17 2023-06-30  2.289373e+07
18 2023-07-31  2.365686e+07
19 2023-08-31  2.365686e+07
20 2023-09-30  2.289373e+07
21 2023-10-31  2.375225e+07
22 2023-11-30  2.289373e+07
23 2023-12-31  2.365686e+07
24 2024-01-31  2.365686e+07
25 2024-02-29  2.213061e+07
26 2024-03-31  2.362506e+07
27 2024-04-30  2.289373e+07
28 2024-05-31  2.365686e+07
29 2024-06-30  3.179685e+04


C:\Users\hp\AppData\Local\Temp\ipykernel_13364\3711381828.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_consumption = df_filtered.groupby(pd.Grouper(


Monthly Consumption and Hourly Bill in Euro of each Month:
         Time  Energy (kWh)  Hourly Bill (€)  Price(cent/kWh)  Temperature
0  2021-01-31      2138.776           135.77         6.351622    -9.738038
1  2021-02-28      1999.589           143.11         7.084027   -12.777381
2  2021-03-31      1554.240            74.60         4.755227    -3.680619
3  2021-04-30      1060.946            45.81         4.558654     2.035833
4  2021-05-31       960.209            52.29         5.696895     7.734812
5  2021-06-30       487.973            32.43         6.963349    16.182361
6  2021-07-31       662.879            58.21         9.765914    18.566935
7  2021-08-31       686.614            54.49         8.456485    13.500672
8  2021-09-30      1020.821           106.75        11.069868     7.390694
9  2021-10-31      1249.997            94.58         8.038060     4.971486
10 2021-11-30      1660.833           197.41        10.651190    -2.793750
11 2021-12-31      1664.150           418